In [1]:
import os
import sys
import glob
import shutil
import pandas as pd
import numpy as np
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

OK


In [2]:
sys.path.append('../..')

In [3]:
from lung_cancer.dataset import FilesIndex, Pipeline, Dataset
from lung_cancer import CTImagesMaskedBatch as CTIMB
from lung_cancer import get_crops

### Set up the two datasets

* indices

In [4]:
DIR_CANCER = '/data/lunaset_split/train/cancer/*'
DIR_NCANCER = '/data/lunaset_split/train/ncancer/*'

In [5]:
cancerix = FilesIndex(path=DIR_CANCER, dirs=True)

In [6]:
ncancerix = FilesIndex(path=DIR_NCANCER, dirs=True)

* datasets

In [7]:
cancerset = Dataset(index=cancerix, batch_class=CTIMB)
ncancerset = Dataset(index=ncancerix, batch_class=CTIMB)

### Pipeline

In [52]:
help(get_crops)

Help on function get_luna_crops in module lung_cancer.pipelines.ppl:

get_luna_crops(cancerset, ncancerset, batch_sizes=(10, 10), hu_lims=(-1000, 400))
    Pipeline for generating batches of cancerous and non-cancerous crops from
            ct-scans in chosen proportion.
    
    Args:
        cancerset: dataset of cancerous crops in blosc format.
        ncancerset: dataset of non-cancerous crops in blosc format.
        batch_sizes: seq of len=2, (num_cancer_batches, num_noncancer_batches).
        hu_lims: seq of len=2, representing limits of hu-trimming in normalize_hu-action.
    
    Return:
        lazy-run pipeline, that will generate batches of size = batch_sizes[0] + batch_sizes[1],
            when run without arguments.



In [53]:
# we want a pipeline, that will generate batch with 20 non-cancerous nodules
# and 20 cancerous
ppl = get_crops(cancerset, ncancerset, batch_sizes=(20, 20))

In [57]:
# generate a batch 
# Note that next_batch'ing is performed without args
bch = ppl.next_batch()

In [55]:
# 40 = 20 + 20
len(bch)

40

In [64]:
# first 20 items are non-cancerous
# (though it is possible that a bit of cancer will be on some crops)
np.sum(bch.get(7, 'masks'))

0.0

In [65]:
np.sum(bch.get(17, 'masks'))

0.0

In [66]:
# while the other items are quite cancerous
np.sum(bch.get(27, 'masks'))

108.0

In [68]:
np.sum(bch.get(37, 'masks'))

320.0

### Test generated batches

In [49]:
def plot_arr_slices(height, *arrays, clim=(-1200, 300)):
    fig, axes = plt.subplots(1, len(arrays), figsize=(14, len(arrays)*8))
    
    for arr, i in zip(arrays, range(len(arrays))):
        depth = arr.shape[0]
        n_slice = int(depth * height)
        
        kwargs = dict()
        if np.max(arr) - np.min(arr) > 2.0:
            kwargs.update(clim=clim)
        else:
            kwargs.update(clim=(0, 1))
    
        axes[i].imshow(arr[n_slice], cmap=plt.cm.gray, **kwargs)
    plt.show()

In [50]:
interact(lambda height: plot_arr_slices(height, 
                                        bch.get(21, 'images'),
                                        bch.get(21, 'masks'), clim=(0, 255)),
         height=(0.01, 0.99, 0.01))

A Jupyter Widget

<function __main__.<lambda>>